In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [2]:
DATASET_PATH = r'C:\Users\Burak\Desktop\YMGK\son'

In [3]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


In [4]:
IMG_WIDTH = 64
IMG_HEIGHT = 64

NUM_WORKERS = 1
PER_WORKER_BATCH_SIZE = 64
GLOBAL_BATCH_SIZE = PER_WORKER_BATCH_SIZE * NUM_WORKERS

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATASET_PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  label_mode='categorical',
  batch_size=GLOBAL_BATCH_SIZE)

Found 2486 files belonging to 2 classes.
Using 1989 files for training.


In [6]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATASET_PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  label_mode='categorical',
  batch_size=GLOBAL_BATCH_SIZE)

Found 2486 files belonging to 2 classes.
Using 497 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['good', 'kırık']


In [8]:
num_classes = len(class_names)

In [9]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.map(scale, num_parallel_calls=AUTOTUNE).repeat().cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(scale, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
dist_dataset = strategy.experimental_distribute_dataset(train_ds)

In [12]:
with strategy.scope():
    model = Sequential()

    # VGG Blocks
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_WIDTH,IMG_HEIGHT,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Dense layers
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=["accuracy"])

In [13]:
es = EarlyStopping(monitor='loss', verbose=1, mode='min', patience = 2, min_delta=0.01)

In [14]:
history = model.fit(dist_dataset,
            epochs=15,
            steps_per_epoch = 75,
            callbacks=[es])

Epoch 1/15
75/75 [==============================] - 23s 284ms/step - loss: 0.3851 - accuracy: 0.8622
Epoch 2/15
75/75 [==============================] - 22s 294ms/step - loss: 0.2025 - accuracy: 0.9302
Epoch 3/15
75/75 [==============================] - 26s 341ms/step - loss: 0.1570 - accuracy: 0.9494
Epoch 4/15
75/75 [==============================] - 26s 342ms/step - loss: 0.1102 - accuracy: 0.9648
Epoch 5/15
75/75 [==============================] - 28s 371ms/step - loss: 0.0896 - accuracy: 0.9710
Epoch 6/15
75/75 [==============================] - 26s 341ms/step - loss: 0.0540 - accuracy: 0.9840
Epoch 7/15
75/75 [==============================] - 26s 340ms/step - loss: 0.0478 - accuracy: 0.9868
Epoch 8/15
75/75 [==============================] - 25s 339ms/step - loss: 0.0328 - accuracy: 0.9921
Epoch 9/15
75/75 [==============================] - 26s 341ms/step - loss: 0.0409 - accuracy: 0.9885
Epoch 10/15
75/75 [==============================] - 26s 346ms/step - loss: 0.0443 - accura

In [15]:
model.save(r'C:\Users\Burak\Desktop\modelymgk.h5')

In [16]:
model = tf.keras.models.load_model(r'C:\Users\Burak\Desktop\modelymgk.h5')

In [17]:
model.evaluate(test_ds)

8/8 [==============================] - 2s 91ms/step - loss: 0.3681 - accuracy: 0.9517


[0.3680533468723297, 0.9517102837562561]

In [30]:
import keras.utils as image


img = image.load_img(r'C:\Users\Burak\Downloads\10.jpg', target_size = (64, 64))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

deneme =model.predict(img)
prediction = deneme
prediction = prediction[0]

# Öznitelikleri sırasıyla yazdırın
for index, value in enumerate(prediction):
  print(f"{class_names[index]}: {value}")


1/1 [==============================] - 0s 22ms/step
10: 1.0
100: 0.0
20: 0.0
200: 0.0
5: 0.0
50: 0.0
